In [41]:
!pip --q install sentence_transformers torch

In [42]:
from sentence_transformers import SentenceTransformer, util
import torch
import pandas as pd

In [86]:
#load cleaned corpus data
df=pd.read_csv("Data/clean_data.csv")

#Load language model
embedder = SentenceTransformer('all-MiniLM-L6-v2')

# Embed all documents
corpus_embeddings = embedder.encode(df['Sentence'], convert_to_tensor=True)

# Query sentences:
#call data cleaning....
queries = ['What kind of model is used in brill tagger ','why is logistic regression more robust to correlated features', 'How to tackle out of vocabulary']


In [87]:
# Find the closest 5 sentences of the corpus for each query sentence based on cosine similarity
top_k = 5
for query in queries:
    query_embedding = embedder.encode(query, convert_to_tensor=True)

    # We use cosine-similarity and torch.topk to find the highest 5 scores
    cos_scores = util.cos_sim(query_embedding, corpus_embeddings)[0]
    top_results = torch.topk(cos_scores, k=top_k)

    print("\n\n======================\n\n")
    print("Query:", query)
    print("\nTop most similar posts:")
    posts_seen=set()
    for score, idx in zip(top_results[0], top_results[1]):
      #Skip, if this post is already recommended
      post=df.iloc[idx.item()]['Post']
      # if post not in posts_seen:
      #   posts_seen.add(post)
      print("\nPost: {}\nDocument: {}\nScore: {:.4f}".format(post,df.iloc[idx.item()]['Sentence'],score))





Query: What kind of model is used in brill tagger 

Top most similar posts:

Post: 506
Document: is brill tagger a discriminative model or generative model?.
Score: 0.7269

Post: 506
Document: neither. the model does not output probabilities of tags or sequences of tags.
Score: 0.4479

Post: 528
Document: it is a tradeoff of runtime and accuracy. my best recommendation is to try it both ways: implement your model with and without smoothing on the emissions, measure runtime and accuracy for both options, and decide if the tradeoff is worthwhile. looking at the theory, smoothing the emission probabilities is expected to cause a substantial increase in runtime, because now you are calculating paths into every tag for every word, instead of throwing away most of the paths, it also has the potential for an improvement in accuracy, since you may run into a situation where context strongly suggests a tag but the current word was only seen with other tags. however, the improvement in accur